In [ ]:
import math
import logging
import pandas as pd
import altair as alt
import numpy as np
from scipy import ndimage
from astropy.convolution import Gaussian2DKernel

import sys,os,os.path
sys.path.append(os.path.expanduser('~/src/spinorama/src'))

from spinorama.load import parse_all_speakers, parse_graphs_speaker, graph_melt
from spinorama.graph import contour_params_default

#df = parse_graphs_speaker('Adam', 'Adam S2V', 'klippel')
# df = parse_graphs_speaker('Klipsch', 'Klipsch R41M', 'klippel')
df = parse_graphs_speaker('Genelec', 'Genelec 8030A', 'princeton')
# print(df)
logging.basicConfig(level='DEBUG')

In [ ]:
def compute_contour(dfu):
    vrange = []
    # normalize dB values wrt on axis                                                                                                                 
    dfm = dfu.copy()
    for c in dfu.columns:
        if c != 'Freq' and c != 'On Axis':
            dfm[c] = dfu[c] - dfu['On Axis']
            angle = int(c[:-1])
            vrange.append(angle)
        if c == 'On Axis':
            vrange.append(0)
    dfm['On Axis'] = 0

    # melt                                                                                                                                            
    dfm = graph_melt(dfm)
    # compute numbers of measurements                                                                                                                 
    nm = dfm.Measurements.nunique()
    nf = int(len(dfm.index) / nm)
    # print('unique={:d} nf={:d}'.format(nm,nf))
    # index grid on a log scale log 2 ±= 0.3                                                                                                          
    hrange = np.floor(np.logspace(1.3, 4.3, nf))
    #hrange = np.linspace(20,20000,nf)
    # print(vrange)                                                                                                                                   
    # sort data per experiments (not usefull for DataFrame but for 2d array)                                                                          
    # anglemin = np.array(vrange).min()/10                                                                                                            
    # perm = [int(vrange[i]/10-anglemin) for i in range(0, len(vrange))]                                                                              
    # pvrange = [vrange[perm[i]] for i in range(0,len(vrange))]                                                                                       
    # 3d mesh    
    # print(hrange, vrange)
    af, am = np.meshgrid(hrange, vrange)
    # print(af, am)
    # since it is melted generate slices                                                                                                              
    az = np.array([dfm.dB[nf * i:nf * (i + 1)] for i in range(0, nm)])
    # smooth values to .1                                                                                                                             
    # az = np.floor(az*10)/10                                                                                                                         
    return (af, am, az)



In [ ]:
ykeep = [20, 30, 100, 200, 300, 400, 500,
             1000, 2000, 3000, 4000, 5000,
             10000, 20000]

def reshape(x, y, z, nscale):
    nx, ny = x.shape
    # linear axis                                                                                                                                     
    lx = np.linspace(np.min(y), np.max(y), nx*nscale)
    # print(lx)
    # this is not, interpolate between each point                                                                                                     
    lyi = [np.linspace(x[0][i], x[0][i+1], nscale, endpoint=False)
           for i in range(0, len(x[0])-1)]
    # flatten then pad with the last value                                                                                                            
    ly = [i for j in lyi for i in j] + \
        [x[0][len(x[0])-1] for i in range(0, nscale)]
    ykeep = [20, 30, 100, 200, 300, 400, 500,
             1000, 2000, 3000, 4000, 5000,
             10000, 20000]
    def close(i, ykeep):
        for z in ykeep:
            if abs((i-z)/z) < 0.01:
                ykeep.remove(z)
                return z
        return i
    ly = [ close(i, ykeep) for i in ly]
    # build the mesh (reverse order)                                                                                                                  
    rx, ry = np.meshgrid(ly, lx)
    # copy paste the values of z into rz                                                                                                              
    rz = np.repeat(np.repeat(z, nscale, axis=1), nscale, axis=0)
    return (rx, ry, rz)


def compute_contour_smoothed(dfu):
    # compute contour                                                                                                                                 
    x, y, z = compute_contour(dfu)
    # std_dev = 1                                                                                                                                     
    kernel = Gaussian2DKernel(1, mode='oversample', factor=10)
    # extend array by x5                                                                                                                              
    rx, ry, rz = reshape(x, y, z, 5)
    # convolve with kernel                                                                                                                            
    rzs = ndimage.convolve(rz, kernel.array, mode='mirror')
    # return                                                                                                                                          
    return (rx, ry, rzs)

In [ ]:
xTicks = [i*10 for i in range(2,10)] + [i*100 for i in range(1,10)] + [i*1000 for i in range(1, 21)]
yTicks = [-180+10*i for i in range(0,37)]

def graph_contour_common(df, transformer, graph_params):
    try:
        width = graph_params['width']
        height = graph_params['height']
        # more interesting to look at -3/0 range                                                                                                      
        speaker_scale = None
        if 'contour_scale' in graph_params.keys():
            speaker_scale = graph_params['contour_scale']
        else:
            speaker_scale = contour_params_default['contour_scale']
        af, am, az = transformer(df)
        freq = af.ravel()
        angle = am.ravel()
        db = az.ravel()
        if (freq.size != angle.size) or (freq.size != db.size):
            print('Contour: Size freq={:d} angle={:d} db={:d}'.format(freq.size, angle.size, db.size))
            return None
        source = pd.DataFrame({'Freq': freq, 'Angle': angle, 'dB': db})
        
        return alt.Chart(source).mark_point(
        ).transform_filter(
            'datum.Freq>400'
        ).encode(
            alt.X('Freq:O', 
                  scale=alt.Scale(type="log"),
                  axis=alt.Axis(
                    format='.0s', 
                    values=xTicks,
                    labelAngle=0,
                    title='Freq (Hz)',
                    labelExpr="datum.value % 100 ? null : datum.label")),
            alt.Y('Angle:O', 
                  axis=alt.Axis(
                    format='.0d', 
                      values=yTicks,
                      title='Angle',
                    labelExpr="datum.value % 30 ? null : datum.label")),
            alt.Color('dB:Q', 
                      scale=alt.Scale(scheme='lightmulti', domain=speaker_scale, nice=True))
        ).properties(
            width=width,
            height=height
        )
    except KeyError as ke:
        print('Failed with {0}'.format(ke))
        return None


def graph_contour(df, graph_params):
    return graph_contour_common(df, compute_contour, graph_params)


def graph_contour_smoothed(df, graph_params):
    return graph_contour_common(df, compute_contour_smoothed, graph_params)

dfu = df['SPL Vertical_unmelted']
g = graph_contour(dfu, contour_params_default)
gs = graph_contour_smoothed(dfu, contour_params_default)
g & gs

In [ ]:
 x, y, z = compute_contour(dfu)

In [ ]:
rx, ry, rz = reshape(x, y, z, 5)

In [ ]:
np.min(rx), np.max(rx)
np.min(ry), np.max(ry)
np.min(rz), np.max(rz)

In [ ]:
kernel = Gaussian2DKernel(1, mode='oversample', factor=10)

In [ ]:
rzs = ndimage.convolve(rz, kernel.array, mode='mirror')
np.min(rzs), np.max(rzs)